In [1]:
class MulLayer: #곱셈계층
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y 
        
        return out
    
    def backward(self, dout): #dout으로 들어가는 인수는 '순전파의 출력에 대한 미분'
        dx = dout * self.y #x와 y를 바꾼다
        dy = dout * self.x 
        
        return dx, dy

In [2]:
#forward EX
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
#back EX
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [4]:
class AddLayer: #덧셈계층
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1 
        dy = dout * 1
        return dx, dy

In [5]:
#사과 귤 예제 잠시 스킵

In [6]:
class Relu: #ReLu 구현
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
    
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

In [8]:
#어파인 변환 : 신경망의 순전파 떄 수행하는 행렬의 곱
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

In [9]:
#softmax-with-loss
class softmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None    # softmax의 출력
        self.x = None    #정답 레이블
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [10]:
import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

#미니배치를 무작위로 선정하기에, 확률적 경사 하강법 SGD라 부른다.
#cs231n에 해당 코드가 나온다 한다.
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #1번째 층의 가중치
        self.params['b1'] = np.zeros(hidden_size)                                      # 1전째 층의 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        #계층 생성
        self.layers = OrderedDict() #순서가 있는 딕셔너리 (추가한 순서대로 각 계층의 forward 메서드만 호출하면 된다)
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
     #x : 입력 데이터 , t: 정답 레이블
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self,x,t): 
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim !=1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self,x, t): #가중치의 매개변수의 기울기를 구한다
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}    #기울기를 보관하는 딕셔너리 변수
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])  # 1번째 층의 가중치의 기울기
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) # 1번째 층의 편향의 기울기
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads
    
    def gradient(self, x, t):
        #순전파
        self.loss(x,t)
        
        #역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        #결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [11]:
#backpropagation으로 구함 기울기 검증
from dataset.mnist import load_mnist

(x_train, t_train),(x_test, t_test) = load_mnist(normalize=True, one_hot_label= True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

#각 가중치의 차이의 절대값을 구한 후 그 절대값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

#아래 결과로 오차가 매우 작은수치라는 것을 확인할 수 있다.

W1:3.9443223747012807e-10
b1:2.2242357069763954e-09
W2:6.084434563192076e-09
b2:1.3937932863278268e-07


In [ ]:
iters_num = 10000 
train_size = x_train.shape[0]
batch_size = 100 
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
        
    #1 에폭당 정확도 계산 
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.17018333333333333 0.1674
0.89985 0.9045
0.9219166666666667 0.9245
0.9370333333333334 0.9366
0.9469833333333333 0.9451
0.9532833333333334 0.9508
0.9568666666666666 0.9557
0.9623 0.9568
0.9638166666666667 0.9593
0.9679666666666666 0.9628
0.9709 0.9627
0.9732166666666666 0.9667
0.9742333333333333 0.9672
0.9764666666666667 0.9694
